In [1]:
import os
import pandas as pd
import sqlite3
from query_insights.utils import read_and_process_data,load_config
from query_insights.db_loading import DatabaseLoading

2023-08-04 15:57:30.653347: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-04 15:57:31.246494: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-04 15:57:31.271780: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-04 15:57:31.271842: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
# Switch data_config.yaml to data_config_azure.yaml to access blob storage
data_config_path = "../../configs/local/data_config.yaml"
user_config_path = "../../configs/local/user_config.yaml"
model_config_path = "../../configs/model_config.yaml"

# config_path = r'D:\Projects\Code_templates\NLP\query_insights\query_insights_generator\ui\gradio\app_files\config\dev\config_ui.yaml'

In [3]:
data_config = load_config(cfg_file=data_config_path)
user_config = load_config(cfg_file=user_config_path)
model_config = load_config(cfg_file=model_config_path)

In [4]:
tables = {
    "invoice_data": "invoice_data_masked.csv",
    "material_desc": "material_descriptions_masked.csv",
    "product_data": "product_data_masked.csv",
    "warehouse_metrics":"warehouse_metrics.csv",
    "warehouse_metrics_monthly":"warehouse_metrics_monthly.csv"
    # "carrier_metrics":"carrier_metrics.csv",
    # "carrier_metrics_monthly":"carrier_metrics_monthly.csv"
}

In [5]:
# tables = data_config.path.input_file_names
database_connection = DatabaseLoading(data_config=data_config, fs_key=None)
conn = database_connection.connection_db()
for table in list(tables.keys()):
    file_name = tables[table]
    df = read_and_process_data(
        path=os.path.join(data_config.path.input_data_path, file_name)
    )
    print(table)
    print(len(df))
    database_connection.load_to_db(df=df, table_name=table)


True
invoice_data
15634
material_desc
2865
product_data
260868
warehouse_metrics
42
warehouse_metrics_monthly
167


In [6]:
conn = sqlite3.connect(data_config.db_params.sqlite_database_path)

query =f"""SELECT COUNT(*) FROM invoice_data"""

cursor = conn.cursor()

cursor.execute(query)

cursor.fetchall()



[(15634,)]